# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import PicklingError
from joblib import dump, load
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, recall_score, classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, matthews_corrcoef, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from datetime import datetime
from tensorflow import keras
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.utils.data import evaluate_print
from IPython.display import display, Markdown
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Settings
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "device_sessions_role"
TIME_BUCKETS  = [
    "7d", 
    "2d", 
    "1d", 
    "12h",
    "6h", 
    "1h"
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def add_count_threat_role(x):
    avg_session_duration = x.session_duration.mean()
    threat = 1 if 1 in x.threat.values else 0
    return pd.Series([len(x), avg_session_duration, threat, x.role.iloc[0]], index=["count", "avg_session_duration", "threat", "role"])


def train_classifier_on_single_users(df, classifier, time_bucket, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in df.groupby("role"):

            # Clustering methods do not work if all counts are zero ("could not form valid cluster separation")
            if group["count"].sum() == 0:
                group["y_true"] = group["threat"]
                group["y_pred"] = 0
                group["scores"] = 0
                timings = pd.Series(
                    {
                        "training_time": 0,
                        "inference_time": 0
                    }
                )
                user_timings = user_timings.append(timings, ignore_index=True)
                user_predictions = pd.concat([user_predictions, group])
                continue

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])


            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group[["count", "avg_session_duration"]])
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group[["count", "avg_session_duration"]])
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, time_bucket, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.time_bucket = time_bucket
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=1
                    )
            except (ValueError, IndexError) as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    """Function that orchestrates the training process.

    Args:
        df (pandas.DataFrame): The dataframe with the training data.
        classifier (sklearn.base.BaseEstimator): The pyod classifier that
            implements the interfaces from the sklearn BaseEstimator class.
        parameters (dict): The parameters that should be tested as part of
            the grid search.

    Returns:
        bool: The return value. True for success, False otherwise.

    """
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, time_bucket in enumerate(TIME_BUCKETS, start=1):

        display(Markdown(f"# {i}/{len(TIME_BUCKETS)} - Bucket: {time_bucket}"))

        # Group the dataframe by users and time buckets
        tmp = df.groupby([df["user"], pd.Grouper(freq=time_bucket, key="date")]).apply(add_count_threat_role)
        tmp = tmp.reindex(pd.MultiIndex.from_product(tmp.index.levels))
        tmp = tmp.fillna(0)

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=tmp,
            classifier=classifier,
            time_bucket=time_bucket,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "device_sessions"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 24 different hyperparameter combinations
 Training Time: 0.4525    Inference Time: 0.1354    pAUC: 0.4876 ± 0.0063   Recall: 0.0134 ± 0.0115   TN: 18832.0   FP: 69.0    FN: 176.0   TP: 3.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.3930    Inference Time: 0.3418    pAUC: 0.4876 ± 0.0063   Recall: 0.0268 ± 0.0124   TN: 18756.0   FP: 145.0   FN: 171.0   TP: 8.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 5.8464    Inference Time: 1.0608    pAUC: 0.4837 ± 0.0033   Recall: 0.0190 ± 0.0067   TN: 18748.0   FP: 153.0   FN: 174.0   TP: 5.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 11.3306   Inference Time: 2.0360    pAUC: 0.4825 ± 0.0024   Recall: 0.0168 ± 0.0050   TN: 18747.0   FP: 154.0   FN: 175.0   TP: 4.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4726    Inference Time: 0.1437    pAUC: 0.4900 ± 0.0069   Recall: 0.0257 ± 0.0120   TN: 18813.0   FP: 88.0    FN: 171.0   TP: 8.0   Param

# 2/6 - Bucket: 2d

Testing 24 different hyperparameter combinations
 Training Time: 0.4923    Inference Time: 0.1496    pAUC: 0.4853 ± 0.0055   Recall: 0.0154 ± 0.0100   TN: 65853.0   FP: 137.0   FN: 252.0   TP: 8.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.4782    Inference Time: 0.4123    pAUC: 0.4853 ± 0.0047   Recall: 0.0223 ± 0.0092   TN: 65618.0   FP: 372.0   FN: 252.0   TP: 8.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 6.8642    Inference Time: 1.3895    pAUC: 0.4839 ± 0.0056   Recall: 0.0200 ± 0.0113   TN: 65618.0   FP: 372.0   FN: 250.0   TP: 10.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 12.7922   Inference Time: 2.6487    pAUC: 0.4811 ± 0.0029   Recall: 0.0146 ± 0.0057   TN: 65615.0   FP: 375.0   FN: 254.0   TP: 6.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4849    Inference Time: 0.1499    pAUC: 0.4902 ± 0.0041   Recall: 0.0277 ± 0.0101   TN: 65825.0   FP: 165.0   FN: 250.0   TP: 10.0   Par

# 3/6 - Bucket: 1d

...

Testing 24 different hyperparameter combinations
 Training Time: 0.5343    Inference Time: 0.1690    pAUC: 0.4833 ± 0.0032   Recall: 0.0143 ± 0.0071   TN: 264315.0   FP: 208.0   FN: 466.0   TP: 11.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.7083    Inference Time: 0.5118    pAUC: 0.4832 ± 0.0047   Recall: 0.0180 ± 0.0091   TN: 263798.0   FP: 725.0   FN: 461.0   TP: 16.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 8.2429    Inference Time: 1.9797    pAUC: 0.4830 ± 0.0026   Recall: 0.0180 ± 0.0051   TN: 263684.0   FP: 839.0   FN: 465.0   TP: 12.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 15.1583   Inference Time: 3.7789    pAUC: 0.4836 ± 0.0021   Recall: 0.0193 ± 0.0041   TN: 263679.0   FP: 844.0   FN: 466.0   TP: 11.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.5334    Inference Time: 0.1687    pAUC: 0.4839 ± 0.0069   Recall: 0.0168 ± 0.0129   TN: 264028.0   FP: 495.0   FN: 458.0   TP: 19

# 5/6 - Bucket: 6h

Testing 24 different hyperparameter combinations
 Training Time: 0.5265    Inference Time: 0.1604    pAUC: 0.4812 ± 0.0033   Recall: 0.0109 ± 0.0073   TN: 528818.0   FP: 386.0   FN: 518.0   TP: 13.0   Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.7188    Inference Time: 0.5192    pAUC: 0.4822 ± 0.0031   Recall: 0.0162 ± 0.0060   TN: 528415.0   FP: 789.0   FN: 517.0   TP: 14.0   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 8.4802    Inference Time: 2.0332    pAUC: 0.4817 ± 0.0021   Recall: 0.0154 ± 0.0040   TN: 528316.0   FP: 888.0   FN: 519.0   TP: 12.0   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 15.5052   Inference Time: 3.8889    pAUC: 0.4811 ± 0.0010   Recall: 0.0143 ± 0.0019   TN: 528298.0   FP: 906.0   FN: 522.0   TP: 9.0   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.5412    Inference Time: 0.1677    pAUC: 0.4840 ± 0.0048   Recall: 0.0177 ± 0.0093   TN: 528796.0   FP: 408.0   FN: 513.0   TP: 18.

# 6/6 - Bucket: 1h

Testing 24 different hyperparameter combinations
 Training Time: 0.5098    Inference Time: 0.1666    pAUC: 0.4761 ± 0.0011   Recall: 0.0034 ± 0.0022   TN: 3176408   FP: 612     FN: 1117   TP: 8     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.8085    Inference Time: 0.5694    pAUC: 0.4772 ± 0.0007   Recall: 0.0066 ± 0.0012   TN: 3175565   FP: 1455    FN: 1116   TP: 9     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 8.7757    Inference Time: 2.3234    pAUC: 0.4780 ± 0.0005   Recall: 0.0082 ± 0.0010   TN: 3175419   FP: 1601    FN: 1114   TP: 11    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 16.4722   Inference Time: 4.4225    pAUC: 0.4778 ± 0.0003   Recall: 0.0078 ± 0.0007   TN: 3175407   FP: 1613    FN: 1115   TP: 10    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.4961    Inference Time: 0.1646    pAUC: 0.4757 ± 0.0005   Recall: 0.0030 ± 0.0009   TN: 3176522   FP: 498     FN: 1120   TP: 5     Params: {'

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 28 different hyperparameter combinations
 Training Time: 0.4079    Inference Time: 0.1201    pAUC: 0.4923 ± 0.0000   Recall: 0.0056 ± 0.0000   TN: 18722    FP: 179     FN: 178   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.5687    Inference Time: 0.1451    pAUC: 0.4872 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 18765    FP: 136     FN: 179   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 0.7186    Inference Time: 0.1693    pAUC: 0.4833 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 18674    FP: 227     FN: 179   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 0.8698    Inference Time: 0.1866    pAUC: 0.4877 ± 0.0000   Recall: 0.0056 ± 0.0000   TN: 18653    FP: 248     FN: 178   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.4171    Inference Time: 0.1167    pAUC: 0.4909 ± 0.0000   Recall: 0.0056 ± 0.0000   TN: 18712    FP: 189     FN: 178   TP: 1     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Bucket: 2d

Testing 28 different hyperparameter combinations
 Training Time: 0.4921    Inference Time: 0.1491    pAUC: 0.5262 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65311    FP: 679     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.6797    Inference Time: 0.2107    pAUC: 0.5323 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65411    FP: 579     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 0.8781    Inference Time: 0.2366    pAUC: 0.5315 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65356    FP: 634     FN: 260   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.0766    Inference Time: 0.2873    pAUC: 0.5397 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65349    FP: 641     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.4838    Inference Time: 0.1619    pAUC: 0.5612 ± 0.0000   Recall: 0.0192 ± 0.0000   TN: 65450    FP: 540     FN: 255   TP: 5     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Bucket: 1d

...

Testing 28 different hyperparameter combinations
 Training Time: 0.5653    Inference Time: 0.2006    pAUC: 0.5841 ± 0.0000   Recall: 0.0629 ± 0.0000   TN: 261912   FP: 2611    FN: 447   TP: 30    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.8071    Inference Time: 0.2837    pAUC: 0.6132 ± 0.0000   Recall: 0.0713 ± 0.0000   TN: 261874   FP: 2649    FN: 443   TP: 34    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 1.5010    Inference Time: 0.5516    pAUC: 0.6149 ± 0.0000   Recall: 0.0901 ± 0.0000   TN: 261921   FP: 2602    FN: 434   TP: 43    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.3172    Inference Time: 0.4505    pAUC: 0.6399 ± 0.0000   Recall: 0.0734 ± 0.0000   TN: 261891   FP: 2632    FN: 442   TP: 35    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.5506    Inference Time: 0.2153    pAUC: 0.5796 ± 0.0000   Recall: 0.0398 ± 0.0000   TN: 261280   FP: 3243    FN: 458   TP: 19    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Bucket: 6h

Testing 28 different hyperparameter combinations
 Training Time: 0.5560    Inference Time: 0.2012    pAUC: 0.6414 ± 0.0000   Recall: 0.0678 ± 0.0000   TN: 523971   FP: 5233    FN: 495   TP: 36    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.8195    Inference Time: 0.2898    pAUC: 0.6824 ± 0.0000   Recall: 0.0904 ± 0.0000   TN: 523953   FP: 5251    FN: 483   TP: 48    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 1.0958    Inference Time: 0.3832    pAUC: 0.6839 ± 0.0000   Recall: 0.0847 ± 0.0000   TN: 524048   FP: 5156    FN: 486   TP: 45    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.3616    Inference Time: 0.4705    pAUC: 0.6913 ± 0.0000   Recall: 0.1243 ± 0.0000   TN: 524011   FP: 5193    FN: 465   TP: 66    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.6905    Inference Time: 0.2778    pAUC: 0.6728 ± 0.0000   Recall: 0.0772 ± 0.0000   TN: 524036   FP: 5168    FN: 490   TP: 41    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Bucket: 1h

Testing 28 different hyperparameter combinations
 Training Time: 0.6415    Inference Time: 0.2625    pAUC: 0.8535 ± 0.0000   Recall: 0.0818 ± 0.0000   TN: 3161023   FP: 15997   FN: 1033   TP: 92    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.9813    Inference Time: 0.4492    pAUC: 0.8486 ± 0.0000   Recall: 0.1804 ± 0.0000   TN: 3141890   FP: 35130   FN: 922   TP: 203   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 1.3630    Inference Time: 0.5690    pAUC: 0.8628 ± 0.0000   Recall: 0.1947 ± 0.0000   TN: 3143654   FP: 33366   FN: 906   TP: 219   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 1.7344    Inference Time: 0.7201    pAUC: 0.8593 ± 0.0000   Recall: 0.2000 ± 0.0000   TN: 3144434   FP: 32586   FN: 900   TP: 225   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.6569    Inference Time: 0.2682    pAUC: 0.8700 ± 0.0000   Recall: 0.3111 ± 0.0000   TN: 3144499   FP: 32521   FN: 775   TP: 350   Params: {'n_bins': 8, 'n_random_cuts'

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.2808    Inference Time: 0.1326    pAUC: 0.5475 ± 0.0000   Recall: 0.0615 ± 0.0000   TN: 18721    FP: 180     FN: 168   TP: 11    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5475 ± 0.0000
Recall: 0.0615 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.2967    Inference Time: 0.1551    pAUC: 0.5771 ± 0.0000   Recall: 0.1000 ± 0.0000   TN: 65353    FP: 637     FN: 234   TP: 26    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5771 ± 0.0000
Recall: 0.1000 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.3284    Inference Time: 0.2217    pAUC: 0.5969 ± 0.0000   Recall: 0.0922 ± 0.0000   TN: 261917   FP: 2606    FN: 433   TP: 44    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5969 ± 0.0000
Recall: 0.0922 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.3416    Inference Time: 0.2373    pAUC: 0.6780 ± 0.0000   Recall: 0.1205 ± 0.0000   TN: 523971   FP: 5233    FN: 467   TP: 64    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6780 ± 0.0000
Recall: 0.1205 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 0.3841    Inference Time: 0.3125    pAUC: 0.8577 ± 0.0000   Recall: 0.1929 ± 0.0000   TN: 3145426   FP: 31594   FN: 908   TP: 217   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8577 ± 0.0000
Recall: 0.1929 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.2881    Inference Time: 0.1267    pAUC: 0.5864 ± 0.0000   Recall: 0.0782 ± 0.0000   TN: 18725    FP: 176     FN: 165   TP: 14    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5864 ± 0.0000
Recall: 0.0782 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.3050    Inference Time: 0.1612    pAUC: 0.5996 ± 0.0000   Recall: 0.1077 ± 0.0000   TN: 65356    FP: 634     FN: 232   TP: 28    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5996 ± 0.0000
Recall: 0.1077 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.3312    Inference Time: 0.2180    pAUC: 0.6008 ± 0.0000   Recall: 0.0881 ± 0.0000   TN: 261915   FP: 2608    FN: 435   TP: 42    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6008 ± 0.0000
Recall: 0.0881 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.3379    Inference Time: 0.2277    pAUC: 0.6723 ± 0.0000   Recall: 0.1262 ± 0.0000   TN: 523976   FP: 5228    FN: 464   TP: 67    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6723 ± 0.0000
Recall: 0.1262 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 0.3808    Inference Time: 0.3176    pAUC: 0.8628 ± 0.0000   Recall: 0.2009 ± 0.0000   TN: 3145439   FP: 31581   FN: 899   TP: 226   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8628 ± 0.0000
Recall: 0.2009 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [1, 2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 6/1 - Bucket: 1h

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.8306    Inference Time: 0.2595    pAUC: 0.8665 ± 0.0000   Recall: 0.2311 ± 0.0000   TN: 3145192.0   FP: 31828.0   FN: 865.0   TP: 260.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.8817    Inference Time: 0.2794    pAUC: 0.8671 ± 0.0000   Recall: 0.2142 ± 0.0000   TN: 3145492.0   FP: 31528.0   FN: 884.0   TP: 241.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.3077    Inference Time: 0.3204    pAUC: 0.8651 ± 0.0001   Recall: 0.2080 ± 0.0000   TN: 3145480.0   FP: 31540.0   FN: 891.0   TP: 234.0   

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 168 different hyperparameter combinations
 Training Time: 0.3446    Inference Time: 0.1189    pAUC: 0.5339 ± 0.0000   Recall: 0.0726 ± 0.0000   TN: 18724    FP: 177     FN: 166   TP: 13    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3457    Inference Time: 0.1197    pAUC: 0.5339 ± 0.0000   Recall: 0.0726 ± 0.0000   TN: 18724    FP: 177     FN: 166   TP: 13    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3366    Inference Time: 0.1171    pAUC: 0.5358 ± 0.0000   Recall: 0.0726 ± 0.0000   TN: 18724    FP: 177     FN: 166   TP: 13    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3421    Inference Time: 0.1181    pAUC: 0.5358 ± 0.0000   Recall: 0.0726 ± 0.0000   TN: 18724    FP: 177     FN: 166   TP: 13    Params: {'n_components': 1, 'stan

# 2/6 - Bucket: 2d

Testing 168 different hyperparameter combinations
 Training Time: 0.3395    Inference Time: 0.1167    pAUC: 0.5635 ± 0.0000   Recall: 0.1115 ± 0.0000   TN: 65357    FP: 633     FN: 231   TP: 29    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3395    Inference Time: 0.1160    pAUC: 0.5635 ± 0.0000   Recall: 0.1115 ± 0.0000   TN: 65357    FP: 633     FN: 231   TP: 29    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3436    Inference Time: 0.1179    pAUC: 0.5642 ± 0.0000   Recall: 0.1154 ± 0.0000   TN: 65359    FP: 631     FN: 230   TP: 30    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3351    Inference Time: 0.1144    pAUC: 0.5642 ± 0.0000   Recall: 0.1154 ± 0.0000   TN: 65359    FP: 631     FN: 230   TP: 30    Params: {'n_components': 1, 'stan

# 3/6 - Bucket: 1d

Testing 168 different hyperparameter combinations
 Training Time: 0.3829    Inference Time: 0.1301    pAUC: 0.6110 ± 0.0000   Recall: 0.2000 ± 0.0000   TN: 130948   FP: 1267    FN: 228   TP: 57    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3465    Inference Time: 0.1214    pAUC: 0.6110 ± 0.0000   Recall: 0.2000 ± 0.0000   TN: 130948   FP: 1267    FN: 228   TP: 57    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3571    Inference Time: 0.1252    pAUC: 0.6080 ± 0.0000   Recall: 0.2000 ± 0.0000   TN: 130949   FP: 1266    FN: 228   TP: 57    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3460    Inference Time: 0.1208    pAUC: 0.6080 ± 0.0000   Recall: 0.2000 ± 0.0000   TN: 130949   FP: 1266    FN: 228   TP: 57    Params: {'n_components': 1, 'stan

# 4/6 - Bucket: 12h

Testing 168 different hyperparameter combinations
 Training Time: 0.4033    Inference Time: 0.1395    pAUC: 0.6368 ± 0.0000   Recall: 0.1321 ± 0.0000   TN: 261935   FP: 2588    FN: 414   TP: 63    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3512    Inference Time: 0.1209    pAUC: 0.6368 ± 0.0000   Recall: 0.1321 ± 0.0000   TN: 261935   FP: 2588    FN: 414   TP: 63    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3570    Inference Time: 0.1246    pAUC: 0.6365 ± 0.0000   Recall: 0.1300 ± 0.0000   TN: 261938   FP: 2585    FN: 415   TP: 62    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3609    Inference Time: 0.1255    pAUC: 0.6365 ± 0.0000   Recall: 0.1300 ± 0.0000   TN: 261938   FP: 2585    FN: 415   TP: 62    Params: {'n_components': 1, 'stan

# 5/6 - Bucket: 6h

Testing 168 different hyperparameter combinations
 Training Time: 0.3504    Inference Time: 0.1200    pAUC: 0.7234 ± 0.0000   Recall: 0.1563 ± 0.0000   TN: 523993   FP: 5211    FN: 448   TP: 83    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}


# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 6/1 - Bucket: 1h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 202.6066   Inference Time: 8.6687    pAUC: 0.8679 ± 0.0006   Recall: 0.1790 ± 0.0077   TN: 3145448.0   FP: 31572.0   FN: 915.0   TP: 210.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 143.8567   Inference Time: 7.7744    pAUC: 0.8678 ± 0.0001   Recall: 0.1740 ± 0.0034   TN: 3145801   FP: 31219   FN: 932   TP: 193   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 148.8137   Inference Time: 8.0955    pAUC: 0.8688 ± 0.0020   Recall: 0.1829 ± 0.0082   TN: 3145449   FP: 31571   FN: 913   TP: 212   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training T

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 3/4 - Bucket: 1d

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 94.6250   Inference Time: 3.7769    pAUC: 0.5565 ± 0.0153   Recall: 0.0561 ± 0.0152   TN: 130911.0   FP: 1304.0   FN: 264.0   TP: 21.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=a

# 4/4 - Bucket: 12h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 99.4350   Inference Time: 4.5450    pAUC: 0.5747 ± 0.0076   Recall: 0.0335 ± 0.0113   TN: 261892.0   FP: 2631.0   FN: 458.0   TP: 19.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_53" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_53/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_53/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • 

# 5/4 - Bucket: 6h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 97.2490   Inference Time: 4.6873    pAUC: 0.6117 ± 0.0400   Recall: 0.0795 ± 0.0218   TN: 524066.0   FP: 5138.0   FN: 481.0   TP: 50.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_104" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_104/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_104/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
 

# 6/4 - Bucket: 1h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 125.9703   Inference Time: 7.5942    pAUC: 0.8625 ± 0.0078   Recall: 0.1717 ± 0.0284   TN: 3145719   FP: 31301   FN: 868   TP: 257   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_1915" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_1915/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_1915/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  